In [41]:
# Celda 1
# Imports y lectura de datos

import pandas as pd
import numpy as np
from math import radians, sin, cos, asin, sqrt

from pyomo.environ import (
    ConcreteModel, Set, Param, Var, NonNegativeReals, Binary,
    Objective, Constraint, SolverFactory, value
)

# Rutas de los archivos (ajusta si tus archivos están en otra carpeta)
PATH_CLIENTS = "./data/clients.csv"
PATH_VEHICLES = "./data/vehicles.csv"
PATH_DEPOTS = "./data/depots.csv"
PATH_PARAMS = "./data/parameters_urban.csv"

clients_df = pd.read_csv(PATH_CLIENTS)
vehicles_df = pd.read_csv(PATH_VEHICLES)
depots_df = pd.read_csv(PATH_DEPOTS)
params_df = pd.read_csv(PATH_PARAMS)

print("Clientes:", clients_df.shape)
print("Vehículos:", vehicles_df.shape)
print("Depósitos:", depots_df.shape)
print("Parámetros:", params_df.shape)

clients_df.head(), vehicles_df.head(), depots_df.head(), params_df


Clientes: (90, 7)
Vehículos: (45, 5)
Depósitos: (12, 6)
Parámetros: (10, 4)


(   ClientID StandardizedID  LocationID  Demand  Longitude  Latitude  \
 0         1           C001          12      13 -74.150806  4.679769   
 1         2           C002          13      15 -74.017996  4.733831   
 2         3           C003          14      16 -74.143655  4.620928   
 3         4           C004          15      22 -74.034583  4.733074   
 4         5           C005          16      14 -74.055887  4.820900   
 
   VehicleSizeRestriction  
 0            light truck  
 1            light truck  
 2            light truck  
 3            light truck  
 4             medium van  ,
    VehicleID StandardizedID  Capacity  Range  VehicleType
 0          1           V001       132    146  light truck
 1          2           V002       136    196   medium van
 2          3           V003       115    143  light truck
 3          4           V004       158    174   medium van
 4          5           V005       109    167   medium van,
    DepotID StandardizedID  LocationID  Lo

In [42]:
# Celda 2
# Definición de nodos (depósitos + clientes) y función de distancia

# Unificamos nodos con un ID estándar
# Prefijo "CD" para depósitos y "C" para clientes

depots_nodes = depots_df[["StandardizedID", "Longitude", "Latitude"]].copy()
depots_nodes.rename(columns={"StandardizedID": "NodeID"}, inplace=True)
depots_nodes["NodeType"] = "depot"

clients_nodes = clients_df[["StandardizedID", "Longitude", "Latitude"]].copy()
clients_nodes.rename(columns={"StandardizedID": "NodeID"}, inplace=True)
clients_nodes["NodeType"] = "client"

nodes_df = pd.concat([depots_nodes, clients_nodes], ignore_index=True)

print("Total nodos:", nodes_df.shape[0])
nodes_df.head()


Total nodos: 102


,NodeID,Longitude,Latitude,NodeType
0,CD01,-74.081242,4.750212,depot
1,CD02,-74.109934,4.536383,depot
2,CD03,-74.038548,4.792926,depot
3,CD04,-74.067069,4.721678,depot
4,CD05,-74.138263,4.607707,depot


In [43]:
# Celda 3
# Función haversine y matriz de distancias

def haversine_km(lon1, lat1, lon2, lat2):
    """
    Calcula la distancia en km entre dos puntos (lon, lat) usando la fórmula de Haversine.
    """
    # convertir a radianes
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    R = 6371  # Radio de la Tierra en km
    return R * c

# Creamos la matriz de distancias
node_ids = list(nodes_df["NodeID"])
dist_matrix = pd.DataFrame(index=node_ids, columns=node_ids, dtype=float)

for i in node_ids:
    lon_i = float(nodes_df.loc[nodes_df["NodeID"] == i, "Longitude"].iloc[0])
    lat_i = float(nodes_df.loc[nodes_df["NodeID"] == i, "Latitude"].iloc[0])
    for j in node_ids:
        if i == j:
            dist_matrix.loc[i, j] = 0.0
        else:
            lon_j = float(nodes_df.loc[nodes_df["NodeID"] == j, "Longitude"].iloc[0])
            lat_j = float(nodes_df.loc[nodes_df["NodeID"] == j, "Latitude"].iloc[0])
            dist_matrix.loc[i, j] = haversine_km(lon_i, lat_i, lon_j, lat_j)

dist_matrix.round(2).head()


,CD01,CD02,CD03,CD04,CD05,CD06,CD07,CD08,CD09,CD10,...,C081,C082,C083,C084,C085,C086,C087,C088,C089,C090
CD01,0.00,23.99,6.70,3.54,17.06,12.06,14.36,8.53,2.23,22.42,...,8.63,8.39,21.38,14.64,13.41,22.56,13.55,21.94,18.99,9.79
CD02,23.99,0.00,29.60,21.14,8.53,12.78,9.64,15.85,22.25,1.59,...,18.85,18.29,3.02,10.27,11.07,4.05,14.86,5.14,6.30,15.10
CD03,6.70,29.60,0.00,8.53,23.37,18.46,20.04,14.93,8.94,28.08,...,11.65,11.89,26.79,19.61,19.62,27.60,20.25,28.14,24.02,14.60
CD04,3.54,21.14,8.53,0.00,14.93,10.13,11.54,6.70,3.53,19.60,...,5.20,4.87,18.40,11.40,11.15,19.40,12.66,19.63,15.81,6.43
CD05,17.06,8.53,23.37,14.93,0.00,5.01,4.98,8.53,15.03,7.08,...,14.59,13.81,7.36,7.80,3.78,9.69,6.34,4.94,7.86,10.40


In [44]:
# Celda 4
# Construcción de costos por km por tipo de vehículo

# Parámetros básicos
def get_param(name):
    return float(params_df.loc[params_df["Parameter"] == name, "Value"].iloc[0])

C_fixed = get_param("C_fixed")      # COP/vehículo
C_dist  = get_param("C_dist")       # COP/km
C_time  = get_param("C_time")       # COP/hora
fuel_price = get_param("fuel_price")  # COP/gallon

# Eficiencias por tipo
eff_small_min   = get_param("fuel_efficiency_van_small_min")
eff_small_max   = get_param("fuel_efficiency_van_small_max")
eff_medium_min  = get_param("fuel_efficiency_van_medium_min")
eff_medium_max  = get_param("fuel_efficiency_van_medium_max")
eff_truck_min   = get_param("fuel_efficiency_truck_light_min")
eff_truck_max   = get_param("fuel_efficiency_truck_light_max")

eff_small  = (eff_small_min + eff_small_max) / 2.0
eff_medium = (eff_medium_min + eff_medium_max) / 2.0
eff_truck  = (eff_truck_min + eff_truck_max) / 2.0

# Suposición de velocidad promedio urbana (km/h)
URBAN_SPEED = 25.0

# Costo por km adicional por tipo (combustible + tiempo)
cost_per_km_type = {}

cost_per_km_type["small van"] = C_dist + fuel_price / eff_small + C_time / URBAN_SPEED
cost_per_km_type["medium van"] = C_dist + fuel_price / eff_medium + C_time / URBAN_SPEED
cost_per_km_type["light truck"] = C_dist + fuel_price / eff_truck + C_time / URBAN_SPEED

cost_per_km_type


{'small van': 3211.5, 'medium van': 3347.3333333333335, 'light truck': 3456.0}

In [45]:
# Celda 5
# Tabla de vehículos con costo por km

def map_vehicle_type_to_group(vtype):
    """
    Mapea el tipo de vehículo del CSV a las llaves que usamos en cost_per_km_type.
    Ajusta aquí si los tipos de texto difieren.
    """
    vtype = vtype.strip().lower()
    if "small" in vtype or "van" in vtype and "small" in vtype:
        return "small van"
    if "medium" in vtype or ("van" in vtype and "medium" in vtype):
        return "medium van"
    if "truck" in vtype or "light truck" in vtype:
        return "light truck"
    # fallback: asumimos van mediana
    return "medium van"

vehicles_df["TypeGroup"] = vehicles_df["VehicleType"].apply(map_vehicle_type_to_group)
vehicles_df["CostPerKm"] = vehicles_df["TypeGroup"].apply(lambda t: cost_per_km_type[t])

vehicles_df[["StandardizedID", "VehicleType", "TypeGroup", "Capacity", "Range", "CostPerKm"]].head()


,StandardizedID,VehicleType,TypeGroup,Capacity,Range,CostPerKm
0,V001,light truck,light truck,132,146,3456.000000
1,V002,medium van,medium van,136,196,3347.333333
2,V003,light truck,light truck,115,143,3456.000000
3,V004,medium van,medium van,158,174,3347.333333
4,V005,medium van,medium van,109,167,3347.333333


In [46]:
# Celda 6
# Construcción del modelo Pyomo

model = ConcreteModel()

# Conjuntos
model.V = Set(initialize=list(vehicles_df["StandardizedID"]))          # Vehículos
model.D = Set(initialize=list(depots_df["StandardizedID"]))            # Depósitos
model.C = Set(initialize=list(clients_df["StandardizedID"]))           # Clientes
model.N = Set(initialize=list(nodes_df["NodeID"]))                     # Todos los nodos

# Parámetros de demanda (por cliente)
demand_dict = dict(zip(clients_df["StandardizedID"], clients_df["Demand"]))
model.demand = Param(model.C, initialize=demand_dict, within=NonNegativeReals)

# Capacidades de depósitos
depot_cap_dict = dict(zip(depots_df["StandardizedID"], depots_df["Capacity"]))
model.depot_capacity = Param(model.D, initialize=depot_cap_dict, within=NonNegativeReals)

# Parámetros de vehículos
cap_vehicle_dict = dict(zip(vehicles_df["StandardizedID"], vehicles_df["Capacity"]))
range_vehicle_dict = dict(zip(vehicles_df["StandardizedID"], vehicles_df["Range"]))
cost_km_dict = dict(zip(vehicles_df["StandardizedID"], vehicles_df["CostPerKm"]))

model.cap_vehicle = Param(model.V, initialize=cap_vehicle_dict, within=NonNegativeReals)
model.range_vehicle = Param(model.V, initialize=range_vehicle_dict, within=NonNegativeReals)
model.cost_km = Param(model.V, initialize=cost_km_dict, within=NonNegativeReals)
model.C_fixed = Param(initialize=C_fixed)

# Matriz de distancias como parámetro de Pyomo
dist_dict = {}
for i in node_ids:
    for j in node_ids:
        if i != j:
            dist_dict[(i, j)] = float(dist_matrix.loc[i, j])
        else:
            dist_dict[(i, j)] = 0.0

model.dist = Param(model.N, model.N, initialize=dist_dict, within=NonNegativeReals)

print("Sets y parámetros cargados en Pyomo.")


Sets y parámetros cargados en Pyomo.


In [47]:
# Celda 7
# Variables de decisión básicas
# Variable binaria: vehículo v viaja de i a j
"""
model.x = Var(model.V, model.N, model.N, within=Binary)

# Variable binaria: si el vehículo v es usado
model.y = Var(model.V, within=Binary)

print("Variables x[v,i,j] y y[v] creadas.")
"""

from pyomo.environ import Var, Binary, NonNegativeReals, Constraint, Objective, minimize

# Variables
model.x = Var(model.V, model.N, model.N, within=Binary)   # arco (i,j) usado por v
model.y = Var(model.V, within=Binary)                     # vehículo v se usa o no

# 1. Cada cliente se atiende exactamente una vez
def visit_once_rule(model, c):
    return sum(model.x[v, i, c] for v in model.V for i in model.N if i != c) == 1
model.visit_once = Constraint(model.C, rule=visit_once_rule)

# 2. Conservación de flujo en cada cliente para cada vehículo
def flow_conservation_rule(model, v, c):
    return sum(model.x[v, c, j] for j in model.N if j != c) - \
           sum(model.x[v, i, c] for i in model.N if i != c) == 0
model.flow_conservation = Constraint(model.V, model.C, rule=flow_conservation_rule)

# 3. Salida desde depósitos por vehículo (si se usa)
def start_from_depot_rule(model, v):
    return sum(model.x[v, d, j] for d in model.D for j in model.N if j != d) == model.y[v]
model.start_from_depot = Constraint(model.V, rule=start_from_depot_rule)

# 4. Regreso a depósitos por vehículo (si se usa)
def end_at_depot_rule(model, v):
    return sum(model.x[v, i, d] for d in model.D for i in model.N if i != d) == model.y[v]
model.end_at_depot = Constraint(model.V, rule=end_at_depot_rule)

# 5. Prohibir loops i -> i
def no_loop_rule(model, v, i):
    return model.x[v, i, i] == 0
model.no_loops = Constraint(model.V, model.N, rule=no_loop_rule)


In [48]:
# Celda 8
# Matriz de compatibilidad vehículo–cliente según tamaño

# Definimos niveles de tamaño
size_level = {
    "small van": 1,
    "medium van": 2,
    "light truck": 3,
}

# Nivel por vehículo
veh_size_level = {}
for _, row in vehicles_df.iterrows():
    v = row["StandardizedID"]
    vtype = str(row["VehicleType"]).strip().lower()
    if "small" in vtype:
        group = "small van"
    elif "medium" in vtype:
        group = "medium van"
    elif "truck" in vtype:
        group = "light truck"
    else:
        group = "medium van"  # valor por defecto
    veh_size_level[v] = size_level[group]

# Nivel máximo por cliente
client_max_size = {}
for _, row in clients_df.iterrows():
    c = row["StandardizedID"]
    rest = str(row["VehicleSizeRestriction"]).strip().lower()
    if "small" in rest:
        group = "small van"
    elif "medium" in rest:
        group = "medium van"
    elif "truck" in rest:
        group = "light truck"
    else:
        group = "light truck"  # por defecto, lo más permisivo
    client_max_size[c] = size_level[group]

# Construimos diccionario de compatibilidad
allow_v_c = {}
for v in model.V:
    for c in model.C:
        allow_v_c[(v, c)] = 1 if veh_size_level[v] <= client_max_size[c] else 0

# Lo llevamos a Pyomo como parámetro
model.allow_v_c = Param(model.V, model.C, initialize=allow_v_c)


In [49]:
# Celda 9
# Variables de carga y restricciones básicas de flujo
"""

from pyomo.environ import NonNegativeReals, Reals, Constraint, Objective, minimize

# Variable de carga en cada nodo para cada vehículo
model.load_v = Var(model.V, model.N, within=NonNegativeReals)

# 1. Cada cliente se atiende exactamente una vez
def visit_once_rule(model, c):
    return sum(model.x[v, i, c] for v in model.V for i in model.N if i != c) == 1
model.visit_once = Constraint(model.C, rule=visit_once_rule)

# 2. Conservación de flujo en cada cliente para cada vehículo
def flow_conservation_rule(model, v, c):
    return sum(model.x[v, c, j] for j in model.N if j != c) - \
           sum(model.x[v, i, c] for i in model.N if i != c) == 0
model.flow_conservation = Constraint(model.V, model.C, rule=flow_conservation_rule)

# 3. Salida y regreso a depósitos por vehículo
def start_from_depot_rule(model, v):
    return sum(model.x[v, d, j] for d in model.D for j in model.N if j != d) == model.y[v]
model.start_from_depot = Constraint(model.V, rule=start_from_depot_rule)

def end_at_depot_rule(model, v):
    return sum(model.x[v, i, d] for d in model.D for i in model.N if i != d) == model.y[v]
model.end_at_depot = Constraint(model.V, rule=end_at_depot_rule)

# 4. Prohibir loops i -> i
def no_loop_rule(model, v, i):
    return model.x[v, i, i] == 0
model.no_loops = Constraint(model.V, model.N, rule=no_loop_rule)

# 5. Carga en depósitos es cero
def depot_load_zero_rule(model, v, d):
    return model.load_v[v, d] == 0
model.depot_load_zero = Constraint(model.V, model.D, rule=depot_load_zero_rule)
"""

# Capacidad total por vehículo: suma de demandas de los clientes asignados a v
def vehicle_capacity_rule(model, v):
    return sum(
        model.demand[c] * sum(model.x[v, i, c] for i in model.N if i != c)
        for c in model.C
    ) <= model.cap_vehicle[v]
model.vehicle_capacity = Constraint(model.V, rule=vehicle_capacity_rule)



In [50]:
# Celda 10
# Capacidad y subtours con variable de carga
"""
# 1. Limites de carga en clientes
def load_bounds_rule(model, v, c):
    return model.demand[c] * sum(model.x[v, i, c] for i in model.N if i != c) <= model.load_v[v, c]
model.min_load_client = Constraint(model.V, model.C, rule=load_bounds_rule)

def load_upper_rule(model, v, i):
    return model.load_v[v, i] <= model.cap_vehicle[v]
model.max_load_node = Constraint(model.V, model.N, rule=load_upper_rule)

# 2. Propagación de carga a lo largo de las rutas
def load_propagation_rule(model, v, i, j):
    if i == j:
        return Constraint.Skip
    # Solo tiene sentido cuando j es cliente
    if j not in model.C:
        return Constraint.Skip
    M = model.cap_vehicle[v]
    return model.load_v[v, j] >= model.load_v[v, i] + model.demand[j] - M * (1 - model.x[v, i, j])
model.load_propagation = Constraint(model.V, model.N, model.N, rule=load_propagation_rule)
"""

# 1. Rango máximo por vehículo
def range_rule(model, v):
    return sum(
        model.dist[i, j] * model.x[v, i, j]
        for i in model.N for j in model.N if i != j
    ) <= model.range_vehicle[v]
model.range_limit = Constraint(model.V, rule=range_rule)

# 2. Compatibilidad vehículo–cliente por tamaño
def size_restr_in_rule(model, v, c):
    return sum(model.x[v, i, c] for i in model.N if i != c) <= model.allow_v_c[v, c]
model.size_restr_in = Constraint(model.V, model.C, rule=size_restr_in_rule)

def size_restr_out_rule(model, v, c):
    return sum(model.x[v, c, j] for j in model.N if j != c) <= model.allow_v_c[v, c]
model.size_restr_out = Constraint(model.V, model.C, rule=size_restr_out_rule)


In [51]:
# Celda 10.5
# Vínculo entre uso de vehículo (y) y arcos (x)

# M grande: máximo número de arcos que podría usar un vehículo
# En el peor caso, visita todos los clientes en una sola ruta.
BIG_M = 2 * (len(model.C) + len(model.D))

def link_xy_rule(model, v):
    return sum(
        model.x[v, i, j] for i in model.N for j in model.N if i != j
    ) <= BIG_M * model.y[v]

model.link_xy = Constraint(model.V, rule=link_xy_rule)


In [52]:
# Celda 11
# Rango máximo y compatibilidad vehículo–cliente

# 1. Rango máximo de cada vehículo
def range_rule(model, v):
    return sum(model.dist[i, j] * model.x[v, i, j] for i in model.N for j in model.N if i != j) \
           <= model.range_vehicle[v]
model.range_limit = Constraint(model.V, rule=range_rule)

# 2. Restricción de tamaño: si allow_v_c[v,c] = 0, ese vehículo no puede usar arcos que entren o salgan del cliente
def size_restr_in_rule(model, v, c):
    return sum(model.x[v, i, c] for i in model.N if i != c) <= model.allow_v_c[v, c]
model.size_restr_in = Constraint(model.V, model.C, rule=size_restr_in_rule)

def size_restr_out_rule(model, v, c):
    return sum(model.x[v, c, j] for j in model.N if j != c) <= model.allow_v_c[v, c]
model.size_restr_out = Constraint(model.V, model.C, rule=size_restr_out_rule)


'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [53]:
# Celda 12
# Función objetivo de costo total

def total_cost_rule(model):
    fixed_cost = sum(model.C_fixed * model.y[v] for v in model.V)
    variable_cost = sum(
        model.cost_km[v] * model.dist[i, j] * model.x[v, i, j]
        for v in model.V for i in model.N for j in model.N if i != j
    )
    return fixed_cost + variable_cost

model.TotalCost = Objective(rule=total_cost_rule, sense=minimize)


In [54]:
# Celda 13
# Resolver el modelo

from pyomo.environ import SolverFactory, TerminationCondition

solver = SolverFactory("highs")
solver.options["time_limit"] = 180
solver.options["mip_rel_gap"] = 0.01

result = solver.solve(model, tee=True)

print("Estado:", result.solver.status)
print("Terminación:", result.solver.termination_condition)

tc = result.solver.termination_condition

if tc in [
    TerminationCondition.optimal,
    TerminationCondition.feasible,
    TerminationCondition.maxTimeLimit   # aceptamos mejor solución encontrada
]:
    try:
        print("Costo total (mejor solución encontrada):", model.TotalCost())
    except:
        print("No se pudo evaluar el costo (variables sin asignar).")
else:
    print("El solver no dejó ninguna solución factible usable.")




Running HiGHS 1.12.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 17055 rows; 468225 cols; 3495285 nonzeros; 468225 integer variables (468225 binary)
Coefficient ranges:
  Matrix  [7e-02, 2e+02]
  Cost    [2e+02, 1e+05]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 1e+03]
Presolving model
11134 rows, 321291 cols, 2399009 nonzeros  2s
10446 rows, 245570 cols, 1937279 nonzeros  57s
Presolve reductions: rows 10446(-6609); columns 245570(-222655); nonzeros 1937279(-1558006) 

Solving MIP model with:
   10446 rows
   245570 cols (245570 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   1937279 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivia

In [55]:
# Celda 14
# Resumen de la solución por vehículo

from pyomo.environ import value

veh_rows = []

for v in model.V:
    yv = value(model.y[v])
    if yv < 0.5:
        # vehículo no usado
        continue

    # Demanda atendida por este vehículo
    demand_v = 0.0
    for c in model.C:
        served_c = sum(value(model.x[v, i, c]) for i in model.N if i != c)
        if served_c > 0.5:  # si entra al cliente c
            demand_v += value(model.demand[c])

    # Distancia recorrida por este vehículo
    dist_v = sum(
        value(model.dist[i, j]) * value(model.x[v, i, j])
        for i in model.N for j in model.N if i != j
    )

    veh_rows.append({
        "Vehicle": v,
        "Used": 1,
        "Capacity": value(model.cap_vehicle[v]),
        "DemandServed": demand_v,
        "RangeLimit": value(model.range_vehicle[v]),
        "DistanceUsed": dist_v,
        "SlackCapacity": value(model.cap_vehicle[v]) - demand_v,
        "SlackRange": value(model.range_vehicle[v]) - dist_v,
    })

veh_summary_df = pd.DataFrame(veh_rows)
veh_summary_df


,Vehicle,Used,Capacity,DemandServed,RangeLimit,DistanceUsed,SlackCapacity,SlackRange
0,V001,1,132,93.0,146,73.363018,39.0,72.636982
1,V002,1,136,132.0,196,131.160471,4.0,64.839529
2,V003,1,115,91.0,143,57.137326,24.0,85.862674
3,V004,1,158,69.0,174,63.000606,89.0,110.999394
4,V007,1,126,13.0,93,8.819006,113.0,84.180994
5,V008,1,79,73.0,134,53.277933,6.0,80.722067
6,V010,1,99,44.0,108,45.491857,55.0,62.508143
7,V012,1,98,96.0,716,85.684340,2.0,630.315660
8,V017,1,25,0.0,12,1.585387,25.0,10.414613
9,V020,1,19,12.0,16,3.531530,7.0,12.468470


In [56]:
# Celda 15
# arcos activos en la solución

arc_rows = []

for v in model.V:
    for i in model.N:
        for j in model.N:
            if i == j:
                continue
            x_val = value(model.x[v, i, j])
            if x_val > 0.5:
                arc_rows.append({
                    "Vehicle": v,
                    "From": i,
                    "To": j,
                    "Distance_km": value(model.dist[i, j]),
                })

arcs_df = pd.DataFrame(arc_rows)
arcs_df.sort_values(["Vehicle", "From", "To"], inplace=True)
arcs_df


,Vehicle,From,To,Distance_km
1,V001,C016,CD05,0.710778
2,V001,C020,C042,18.285936
3,V001,C030,C020,9.073933
4,V001,C042,C016,6.335858
5,V001,C044,C057,2.327364
...,...,...,...,...
104,V045,C011,C022,11.014931
105,V045,C022,C026,9.104173
106,V045,C026,CD06,2.739547
107,V045,C072,C011,11.191693


In [57]:
# Celda 16
# Reconstrucción de rutas ordenadas por vehículo

# Mapeo de demanda por cliente para poder usarlo fuera de Pyomo
demand_map = dict(zip(clients_df["StandardizedID"], clients_df["Demand"]))

route_rows = []

for v in arcs_df["Vehicle"].unique():
    sub = arcs_df[arcs_df["Vehicle"] == v].copy()
    if sub.empty:
        continue

    # Construimos mapas de siguiente y anterior
    next_map = {row["From"]: row["To"] for _, row in sub.iterrows()}
    prev_map = {row["To"]: row["From"] for _, row in sub.iterrows()}

    # Buscamos un depósito de inicio: aparece como From pero no como To
    start_candidates = [n for n in next_map.keys()
                        if n.startswith("CD") and n not in prev_map]
    if start_candidates:
        start = start_candidates[0]
    else:
        # Si no encontramos, tomamos cualquier nodo "From" que sea depósito
        depot_from = [n for n in next_map.keys() if n.startswith("CD")]
        start = depot_from[0] if depot_from else list(next_map.keys())[0]

    # Recorremos la ruta
    step = 0
    current = start
    visited = set()

    while current in next_map and (current, next_map[current]) not in visited:
        nxt = next_map[current]
        visited.add((current, nxt))

        # Buscamos el arco concreto
        arc = sub[(sub["From"] == current) & (sub["To"] == nxt)].iloc[0]
        dist = arc["Distance_km"]

        # Demanda atendida en el nodo de llegada
        if nxt.startswith("C"):
            demand_to = demand_map.get(nxt, 0.0)
        else:
            demand_to = 0.0

        route_rows.append({
            "Vehicle": v,
            "Step": step,
            "From": current,
            "To": nxt,
            "Distance_km": dist,
            "DemandAtTo": demand_to,
            "FromIsDepot": 1 if current.startswith("CD") else 0,
            "ToIsDepot": 1 if nxt.startswith("CD") else 0,
        })

        step += 1
        current = nxt

        # Cortamos si regresamos a un depósito distinto al de inicio
        if current.startswith("CD") and current != start and step > 1:
            break

routes_df = pd.DataFrame(route_rows)

# Distancia acumulada y demanda acumulada por vehículo
routes_df["CumDistanceVeh"] = routes_df.groupby("Vehicle")["Distance_km"].cumsum()
routes_df["CumDemandVeh"] = routes_df.groupby("Vehicle")["DemandAtTo"].cumsum()

routes_df.head(20)


,Vehicle,Step,From,To,Distance_km,DemandAtTo,FromIsDepot,ToIsDepot,CumDistanceVeh,CumDemandVeh
0,V001,0,CD05,C045,1.029657,12.0,1,0,1.029657,12.0
1,V001,1,C045,C047,18.665564,12.0,0,0,19.695220,24.0
2,V001,2,C047,C030,14.606564,12.0,0,0,34.301785,36.0
3,V001,3,C030,C020,9.073933,12.0,0,0,43.375718,48.0
4,V001,4,C020,C042,18.285936,12.0,0,0,61.661653,60.0
5,V001,5,C042,C016,6.335858,12.0,0,0,67.997512,72.0
6,V001,6,C016,CD05,0.710778,0.0,0,1,68.708290,72.0
7,V002,0,CD04,C067,1.121908,18.0,1,0,1.121908,18.0
8,V002,1,C067,C060,19.169990,12.0,0,0,20.291898,30.0
9,V002,2,C060,CD05,4.574600,0.0,0,1,24.866498,30.0


In [ ]:
# Celda 17
#  CSV de verificación basado directamente en todos los arcos activos

# 1. Partimos de todos los arcos de la solución
verif_df = arcs_df.copy()   # columnas: Vehicle, From, To, Distance_km

# 2. Agregamos info del vehículo
veh_info = vehicles_df.rename(columns={"StandardizedID": "Vehicle"})[
    ["Vehicle", "VehicleType", "Capacity", "Range"]
]
verif_df = verif_df.merge(veh_info, on="Vehicle", how="left")

# 3. Marcamos si el nodo es depósito
verif_df["FromIsDepot"] = verif_df["From"].str.startswith("CD").astype(int)
verif_df["ToIsDepot"]   = verif_df["To"].str.startswith("CD").astype(int)

# 4. Demanda en el nodo de llegada (solo clientes)
demand_map = dict(zip(clients_df["StandardizedID"], clients_df["Demand"]))
verif_df["DemandAtTo"] = verif_df["To"].map(demand_map).fillna(0.0)

# 5. (Opcional) nos quedamos solo con columnas limpias para el CSV
verif_df_out = verif_df[[
    "Vehicle",
    "VehicleType",
    "Capacity",
    "Range",
    "From",
    "To",
    "FromIsDepot",
    "ToIsDepot",
    "Distance_km",
    "DemandAtTo",
]]

# 6. Guardar CSV
verif_df_out.to_csv("verificacion_caso3.csv", index=False)

verif_df_out.head(20)


,Vehicle,VehicleType,Capacity,Range,From,To,FromIsDepot,ToIsDepot,Distance_km,DemandAtTo
0,V001,light truck,132,146,C016,CD05,0,1,0.710778,0.0
1,V001,light truck,132,146,C020,C042,0,0,18.285936,12.0
2,V001,light truck,132,146,C030,C020,0,0,9.073933,12.0
3,V001,light truck,132,146,C042,C016,0,0,6.335858,12.0
4,V001,light truck,132,146,C044,C057,0,0,2.327364,9.0
5,V001,light truck,132,146,C045,C047,0,0,18.665564,12.0
6,V001,light truck,132,146,C047,C030,0,0,14.606564,12.0
7,V001,light truck,132,146,C057,C044,0,0,2.327364,12.0
8,V001,light truck,132,146,CD05,C045,1,0,1.029657,12.0
9,V002,medium van,136,196,C002,C041,0,0,6.465004,12.0


In [70]:
csv_df = verif_df_out.copy()

print("Filas totales:", len(csv_df))

to_clients = csv_df[csv_df["To"].isin(clients_df["StandardizedID"])]["To"]
print("Clientes distintos:", to_clients.nunique())

print("Demanda en CSV:", csv_df["DemandAtTo"].sum())
print("Demanda total original:", clients_df["Demand"].sum())


Filas totales: 108
Clientes distintos: 90
Demanda en CSV: 1113.0
Demanda total original: 1113
